# Generacion de instancias

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
DEBUG = False
LOCALES = 25
INSTANCIAS = 10

In [20]:
import os

from typing import List
from typing import Tuple

Local = Tuple[int, int]
def save_instance(dataset: str, instance: str, M: int, locales: List[Local]):
    """
    Guarda una instancia con el formato

        n M
        b_1 c_1
        .
        .
        .
        b_n c_n

    Las instancias son tuplas (b_i, c_i)
    """

    # exist_ok para que no falle si ya existe
    os.makedirs(f"instancias/{dataset}", exist_ok=True)

    if DEBUG: print("Escribiendo instancia", f"{os.getcwd()}/instancias/{dataset}/{instance}.in")
    with open(f"instancias/{dataset}/{instance}.in", "w") as f:
        print(len(locales), M, file=f)
        for local in locales:
            print(local[0], local[1], file=f)

## Control

Instancias generadas de forma aleatoria

In [21]:
from typing import Tuple, List
Instance = Tuple[int, List[Local]]

In [22]:
import numpy as np

def randrange(r, n):
    return np.random.randint(low=r[0], high=r[1], size=n)

# rangos
CONTAGIO = (10, 100)
BENEFICIO = (10, 100)
LOCALES = 25    # cantidad de locales
M = 1000  # Una posibilidad es randomizar el M para que sea una instancia realmente de control

def randinst() -> Instance:
    locales = zip(
        randrange(BENEFICIO, LOCALES),
        randrange(CONTAGIO, LOCALES),
    )

    return M, list(locales)
    

In [23]:
for i in range(INSTANCIAS):
    m, locales = randinst()
    save_instance(dataset="control", instance=str(i), M=m, locales=locales)

## Grupos - Solapamiento

In [24]:
BENEFICIO = (10, 50)

def group_instance(cant_grupos: int) -> Instance:
    locales = []
    for l in range(LOCALES):
        c = (l % cant_grupos + 1)
        b = randrange(BENEFICIO, 1)[0]
        locales.append((b, c))

    return 250, locales
    

In [25]:
import random

for n in range(1, LOCALES+1):
    # hay una cantidad de grupos n determinada para cada instancia
    for i in range(INSTANCIAS):
        m, locales = group_instance(n)
        save_instance("grupos", f"{n}-grupos_{i}", m, locales)
    

## One tu rule dem ol

In [26]:
BEN = 10 # ben 10
CONT = 10
BEN_DISTINGUIDO = BEN*LOCALES
M = int(CONT * LOCALES / 2)

for i in range(INSTANCIAS*2):
    locales = [ (BEN, CONT) for _ in range(LOCALES) ]
    locales[i] = (BEN_DISTINGUIDO, CONT)
    save_instance("one-to-rule", "pos_"+str(i), M, locales)


### ¿En qué lugar lo ponemos?

- Podemos variar el lugar en que se coloca el local distinguido.

## Identicos

In [27]:
BEN = 10
CONT = 10
M = int(CONT * LOCALES / 2)

for i in range(INSTANCIAS):
    locales = [ (BEN, CONT) for _ in range(LOCALES) ]
    save_instance("identicos", str(i), M, locales)


## Factibilidad

In [28]:
BENEFICIO = (10,50)
CONTAGIO = (50,100)
M = 200

def fact_instances() -> Instance:
    locales = zip(
        randrange(BENEFICIO, LOCALES),
        randrange(CONTAGIO, LOCALES),
    )

    return M, list(locales)
    

In [29]:
for i in range(INSTANCIAS):
    m, locales = fact_instances()
    save_instance("low-M", str(i), m, locales)
    

## Caché en PD

In [30]:
BENEFICIO = (10,50)
CONTAGIO = (10,500)

def cache_locals() -> List[Local]:
    locales = zip(
        randrange(BENEFICIO, LOCALES),
        randrange(CONTAGIO, LOCALES),
    )

    return list(locales)

In [31]:
# Lo importante de esta experimentación es que no exista solapamiento entre los llamados a la memoria para que se utilicen
# diferentes regiones de la tabla y esto lleve a que se termine desalojando de a una página de memoria (idealmente)  

M = range(1000, 5000, 1000)

for m in M:
    for i in range(INSTANCIAS):
        save_instance("cache", f"{m}-{i}", m, cache_locals())

## Instancias con N variable

In [32]:
N = range(5,35,5) # n de 5, 10, 15, 20, 25 y 30
BENEFICIO = (10,100)
CONTAGIO = (10,100)

def n_instance(n: int) -> Instance:
    locales = zip (
        randrange(BENEFICIO, n),
        randrange(CONTAGIO, n),
    )
    # El M devuelto se obtiene de calcula la media esperada de beneficio (50) dividido dos (2) debido a que se espera que como máximo podemos poner la mitad de los locales
    return n*25, list(locales)

for n in N:
    for i in range(3): # 3 porque sí, algún problema?
        m, locales = n_instance(n)
        save_instance("n-variable", f"{n}-{i}", m, locales)


## Programación Dinámica

In [17]:
N = range(50, 1050, 50)
M = range(50, 1050, 50)
BENEFICIO = (1, 10)
CONTAGIO = (1, 10)

def randinst(n: int) -> List[Local]:
    locales = zip(
        randrange(BENEFICIO, n),
        randrange(CONTAGIO, n),
    )

    return list(locales)

for m in M:
    for n in N:
        for i in range(3):
            save_instance("complejidad-DP", f"{m}-{n}-{i}", m, randinst(n))